<a href="https://colab.research.google.com/github/Nonglassy/Gazua/blob/main/%EA%B2%BD%EC%A0%9C%EC%A7%80%ED%91%9C_%EB%B0%8F_%EA%B8%B0%EC%88%A0%EC%A7%80%ED%91%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#경제지표
import requests
import pandas as pd
import json

def getdata(search_text,st_date,end_date):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36','X-Requested-With':'XMLHttpRequest',}
    data={'search_text':search_text,}
    result1=requests.post('https://www.investing.com/search/service/searchTopBar',data=data,headers=headers)
    data=result1.json()
    first_quote_result=data['quotes'][0]
    curr_id=first_quote_result['pairId'] 
    data={'curr_id':curr_id,'st_date':st_date,'end_date':end_date,'interval_sec':'Daily','action':'historical_data',}
    result2=requests.post('https://www.investing.com/instruments/HistoricalDataAjax',data=data,headers=headers)
    data=result2.text
    data=pd.read_html(data)
    hist_data=data[0]
    return pd.DataFrame(hist_data)


In [36]:
Bitcoin=getdata('bitcoin','01/01/2018','04/30/2022')
TenYear=getdata('united states 10-Year','01/01/2018','04/30/2022')
OVX=getdata('ovx','01/01/2018','04/30/2022')
VIX=getdata('vix','01/01/2018','04/30/2022')

In [37]:
import numpy as np
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

lr=LinearRegression()
imp=IterativeImputer(estimator=lr, tol=1e-10, max_iter=30, verbose=2, imputation_order='roman')

Bitcoin=Bitcoin.drop(Bitcoin.columns[1:],axis=1)
TenYear=TenYear.drop(TenYear.columns[5],axis=1)
OVX=OVX.drop(OVX.columns[5:7],axis=1)
VIX=VIX.drop(VIX.columns[5:7],axis=1)
TenYear.columns=['Date','YOpen','YHigh','YLow','YClose']
OVX.columns=['Date','ovxOpen','ovxHigh','ovxYLow','ovxClose']
VIX.columns=['Date','vixOpen','vixHigh','vixLow','vixClose']

data=pd.merge(left=Bitcoin, right=TenYear, how='outer',on="Date")
data=pd.merge(left=data, right=OVX, how='outer',on="Date")
data=pd.merge(left=data, right=VIX, how='outer',on="Date")
data=data.drop(data.columns[0],axis=1)

data=imp.fit_transform(data)

from datetime import datetime
from pytz import timezone

Otherdata = pd.DataFrame(data, columns=['YOpen','YHigh','YLow','YClose','ovxOpen','ovxHigh','ovxLow','ovxClose','vixOpen','vixHigh','vixLow','vixClose'])
Otherdata=pd.concat([Bitcoin, Otherdata], axis = 1)
Otherdata['Date']=pd.to_datetime(Otherdata['Date'])
Otherdata=Otherdata.sort_values(by=['Date'])
Otherdata=Otherdata.drop(Otherdata.columns[0],axis=1)
Otherdata.reset_index(drop=True, inplace = True)

[IterativeImputer] Completing matrix with shape (1581, 12)
[IterativeImputer] Ending imputation round 1/30, elapsed time 0.02
[IterativeImputer] Change: 8.79915173316312, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 2/30, elapsed time 0.05
[IterativeImputer] Change: 3.501868127243106, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 3/30, elapsed time 0.07
[IterativeImputer] Change: 4.1998925705043675, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 4/30, elapsed time 0.10
[IterativeImputer] Change: 4.13939449895486, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 5/30, elapsed time 0.12
[IterativeImputer] Change: 3.963913444078936, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 6/30, elapsed time 0.15
[IterativeImputer] Change: 3.7810651044078134, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [8]:
#기술지표
!pip install pyupbit
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4075    0  4075    0     0  23285      0 --:--:-- --:--:-- --:--:-- 23419
100  503k  100  503k    0     0  1117k      0 --:--:-- --:--:-- --:--:-- 1117k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4055    0  4055    0     0  21455      0 --:--:-- --:--:-- --:--:-- 21569
100  406k  100  406k    0     0   907k      0 --:--:-- --:--:-- --:--:-- 4025k


In [10]:
import pyupbit
import pandas as pd
import numpy as np
import talib as ta

def getPrice(search,n,finish_date):
    df=pyupbit.get_ohlcv(search,'hour',n,to=finish_date)
    df=df.drop("value",axis=1)
    return pd.DataFrame(df)

def getADX(data):
    adx=ta.ADX(data['high'],data['low'],data['close'],timeperiod=14)
    adx=adx[-1581:]
    return pd.DataFrame(adx)

def getMA50(data):
    MA50=ta.MA(data['close'],timeperiod=50, matype=0)
    MA50=MA50[-1581:]
    return pd.DataFrame(MA50)

def getMACD_signal(data):
    MACD,MACD_signal,MACD_hist=ta.MACD(data['close'],fastperiod=12, slowperiod=26, signalperiod=9)
    MACD_signal=MACD_signal[-1581:]
    return pd.DataFrame(MACD_signal)

BTC=getPrice('KRW-BTC',1631,'20220501')
ETH=getPrice('KRW-ETH',1631,'20220501')
ETC=getPrice('KRW-ETC',1631,'20220501')
BTG=getPrice('KRW-BTG',1631,'20220501')
QTUM=getPrice('KRW-QTUM',1631,'20220501')
XRP=getPrice('KRW-XRP',1631,'20220501')

ADX_BTC=getADX(BTC)
ADX_ETH=getADX(ETH)
ADX_ETC=getADX(ETC)
ADX_BTG=getADX(BTG)
ADX_QTUM=getADX(QTUM)
ADX_XRP=getADX(XRP)

MA50_BTC=getMA50(BTC)
MA50_ETH=getMA50(ETH)
MA50_ETC=getMA50(ETC)
MA50_BTG=getMA50(BTG)
MA50_QTUM=getMA50(QTUM)
MA50_XRP=getMA50(XRP)


MACD_signal_BTC=getMACD_signal(BTC)
MACD_signal_ETH=getMACD_signal(ETH)
MACD_signal_ETC=getMACD_signal(ETC)
MACD_signal_BTG=getMACD_signal(BTG)
MACD_signal_QTUM=getMACD_signal(QTUM)
MACD_signal_XRP=getMACD_signal(XRP)


In [39]:
Fin_BTC=pd.concat([ADX_BTC,MA50_BTC,MACD_signal_BTC],axis=1)
Fin_BTC.columns=['ADX_BTC','MA50_BTC','MACD_signal_BTC']
Fin_ETH=pd.concat([ADX_ETH,MA50_ETH,MACD_signal_ETH],axis=1)
Fin_ETH.columns=['ADX_ETH','MA50_ETH','MACD_signal_ETH']
Fin_ETC=pd.concat([ADX_ETC,MA50_ETC,MACD_signal_ETC],axis=1)
Fin_ETC.columns=['ADX_ETC','MA50_ETC','MACD_signal_ETC']
Fin_BTG=pd.concat([ADX_BTG,MA50_BTG,MACD_signal_BTG],axis=1)
Fin_BTG.columns=['ADX_BTG','MA50_BTG','MACD_signal_BTG']
Fin_QTUM=pd.concat([ADX_QTUM,MA50_QTUM,MACD_signal_QTUM],axis=1)
Fin_QTUM.columns=['ADX_QTUM','MA50_QTUM','MACD_signal_QTUM']
Fin_XRP=pd.concat([ADX_XRP,MA50_XRP,MACD_signal_XRP],axis=1)
Fin_XRP.columns=['ADX_XRP','MA50_XRP','MACD_signal_XRP']

Fin_Last=pd.concat([Fin_BTC,Fin_ETH,Fin_ETC,Fin_BTG,Fin_QTUM,Fin_XRP],axis=1)
Fin_Last.reset_index(drop=True, inplace = True)
print(Fin_Last)

        ADX_BTC    MA50_BTC  MACD_signal_BTC    ADX_ETH   MA50_ETH  \
0     25.704880  15635440.0     1.671972e+06  20.193102   677713.0   
1     25.115793  15890960.0     1.529126e+06  20.874206   694024.0   
2     24.568783  16157140.0     1.417729e+06  21.840361   712234.0   
3     25.034719  16458180.0     1.368570e+06  23.694196   733436.0   
4     26.797373  16832860.0     1.422367e+06  25.885890   757222.0   
...         ...         ...              ...        ...        ...   
1576  20.415702  51910320.0    -6.495258e+05  22.290506  3743220.0   
1577  20.458685  51945140.0    -6.834365e+05  23.140706  3752700.0   
1578  19.237278  51925340.0    -6.948031e+05  22.831687  3760340.0   
1579  18.325504  51941620.0    -7.034052e+05  22.889338  3768240.0   
1580  17.629595  51960980.0    -7.153682e+05  23.242086  3775780.0   

      MACD_signal_ETH    ADX_ETC  MA50_ETC  MACD_signal_ETC    ADX_BTG  \
0       119501.776740  27.858373   32466.0      2947.344014  14.224344   
1       118

In [40]:
#LSTM 모델에 넣을 마지막

Data=pd.concat([Otherdata,Fin_Last],axis=1)
print(Data)

         YOpen    YHigh      YLow    YClose    ovxOpen    ovxHigh     ovxLow  \
0     2.405000  2.40500  2.405000  2.405000  40.267493  40.159137  41.587107   
1     2.461000  2.42900  2.481000  2.416000  22.730000  23.030000  23.180000   
2     2.445000  2.46500  2.472000  2.440000  22.750000  22.730000  24.150000   
3     2.453000  2.45800  2.487000  2.449000  22.120000  23.900000  25.720000   
4     2.476000  2.45800  2.480000  2.433000  22.300000  22.300000  22.480000   
...        ...      ...       ...       ...        ...        ...        ...   
1576  2.728000  2.81800  2.866000  2.724000  53.570000  52.860000  54.130000   
1577  2.826000  2.74000  2.839000  2.715000  53.380000  53.570000  57.000000   
1578  2.832000  2.83400  2.888000  2.796000  51.940000  53.380000  54.790000   
1579  2.938000  2.85400  2.943000  2.831000  53.100000  51.940000  54.720000   
1580  1.872222  1.87114  1.898599  1.843587  42.488965  42.283269  46.579356   

       ovxClose    vixOpen    vixHigh  